In [1]:
#Домашнее задание Титанк

In [259]:
import numpy as np
import pandas as pd

In [260]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [261]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('gender_submission.csv')

In [262]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [263]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [264]:
pre = train[['Name']]
title = pre['Name'].str.split(',').str[1].str.split('.').str[0]
train['Title'] = title
train['Group_num'] = train.Parch + train.SibSp + 1
del pre
del title
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Group_num
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,1


In [265]:
pre = test[['Name']]
title = pre['Name'].str.split(',').str[1].str.split('.').str[0]
test['Title'] = title
test['Group_num'] = train.Parch + train.SibSp + 1
del pre
del title
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Group_num
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs,2
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs,1


In [266]:
train['Age'].fillna((train['Age'].median()), inplace=True)
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
Group_num        0
dtype: int64

In [267]:
test['Age'].fillna((test['Age'].median()), inplace=True)
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Title            0
Group_num        0
dtype: int64

In [268]:
X_train = train[['Pclass','Sex','Age','Title','Group_num']]
X_train.head()

,Pclass,Sex,Age,Title,Group_num
0,3,male,22.0,Mr,2
1,1,female,38.0,Mrs,2
2,3,female,26.0,Miss,1
3,1,female,35.0,Mrs,2
4,3,male,35.0,Mr,1


In [269]:
X_test = test[['Pclass','Sex','Age','Title','Group_num']]
X_test.head()

,Pclass,Sex,Age,Title,Group_num
0,3,male,34.5,Mr,2
1,3,female,47.0,Mrs,2
2,2,male,62.0,Mr,1
3,3,male,27.0,Mr,2
4,3,female,22.0,Mrs,1


In [270]:
from sklearn.tree import DecisionTreeClassifier

In [271]:
from sklearn.preprocessing import LabelEncoder

In [272]:
#le = LabelEncoder()
y = train.Survived
le_sex = LabelEncoder()
X_train.Sex = le_sex.fit_transform(X_train.Sex)
X_test.Sex = le_sex.fit_transform(X_test.Sex)
le_title = LabelEncoder()
X_train.Title = le_title.fit_transform(X_train.Title)
X_test.Title = le_title.fit_transform(X_test.Title)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [273]:
X_train.head()

,Pclass,Sex,Age,Title,Group_num
0,3,1,22.0,11,2
1,1,0,38.0,12,2
2,3,0,26.0,8,1
3,1,0,35.0,12,2
4,3,1,35.0,11,1


In [274]:
X_test.head()

,Pclass,Sex,Age,Title,Group_num
0,3,1,34.5,5,2
1,3,0,47.0,6,2
2,2,1,62.0,5,1
3,3,1,27.0,5,2
4,3,0,22.0,6,1


In [275]:
from sklearn.model_selection import GridSearchCV

In [276]:
depths = np.arange(1,10)
grid = {'max_depth': depths}
gridsearch = GridSearchCV(DecisionTreeClassifier(), grid, scoring='neg_log_loss', cv=5)

In [277]:
%%time
gridsearch.fit(X, y)

Wall time: 202 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [278]:
print('Best Params:')
print(gridsearch.best_params_)
print('Best CV Score:')
print(gridsearch.best_score_)
print('Best CV Estimator:')
print(gridsearch.best_estimator_)

Best Params:
{'max_depth': 2}
Best CV Score:
-0.4762441355281099
Best CV Estimator:
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [279]:
clf_final = gridsearch.best_estimator_

In [280]:
clf_final.fit(X_train, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [281]:
y_test_proba = clf_final.predict_proba(X_test)
y_test = clf_final.predict(X_test)

In [282]:
sample_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [283]:
submit = pd.DataFrame(y_test, columns=sample_submission.columns[1:])
submit['PassengerId'] = sample_submission.PassengerId
submit = submit[[submit.columns[-1]]+list(submit.columns[:-1])]
submit.to_csv('submit.csv', index=False)

In [284]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [285]:
pd.Series(le_sex.inverse_transform(X_test.Sex)).value_counts().sort_index()

female    152
male      266
dtype: int64

In [286]:
print(export_graphviz(clf_final, out_file=None, filled=True, feature_names=list(X_train.columns), class_names=None))

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="Sex <= 0.5\ngini = 0.473\nsamples = 891\nvalue = [549, 342]", fillcolor="#e5813960"] ;
1 [label="Pclass <= 2.5\ngini = 0.383\nsamples = 314\nvalue = [81, 233]", fillcolor="#399de5a6"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="gini = 0.1\nsamples = 170\nvalue = [9, 161]", fillcolor="#399de5f1"] ;
1 -> 2 ;
3 [label="gini = 0.5\nsamples = 144\nvalue = [72, 72]", fillcolor="#e5813900"] ;
1 -> 3 ;
4 [label="Title <= 9.0\ngini = 0.306\nsamples = 577\nvalue = [468, 109]", fillcolor="#e58139c4"] ;
0 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
5 [label="gini = 0.5\nsamples = 53\nvalue = [26, 27]", fillcolor="#399de509"] ;
4 -> 5 ;
6 [label="gini = 0.264\nsamples = 524\nvalue = [442, 82]", fillcolor="#e58139d0"] ;
4 -> 6 ;
}


![alt text](Webgraphviz.jpg "Graf")